In [3]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import ipywidgets
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import model_selection
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm

/home/sandhya/Project/final/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [13]:
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [97]:
%run ../paths.py
CONST = CONST()

In [98]:
train_data = pd.read_csv(CONST.TRAIN_DATA)
test_data = pd.read_csv(CONST.TEST_DATA)

In [99]:
train_data.drop(train_data.columns[0],axis=1,inplace=True)
test_data.drop(test_data.columns[0],axis=1,inplace=True)


In [100]:
train_data.head()

,tweet,class
0,Mannnnnn these hoes aint loyal,1
1,have a fabulous weekend beautiful souls #nofil...,0
2,#fathersday #dad @user,0
3,RT @SheswantstheD: these hoes ain't loyal http...,1
4,newhairâ¨ð #lapis#rollen#new#hair#brond#gr...,0


In [101]:
test_data.head()

,tweet,class
0,What I look like ho? I look like YES and you l...,1
1,@user lots of #resistance and #challenges fro...,0
2,what's wrong with ppl nowadays? i was conducti...,0
3,@Montulos Lakers are trash,1
4,will be playing rf simpson in #singinintherain...,0


In [102]:
vec = TfidfVectorizer(ngram_range=(1,2),max_features=10000)
train_tfidf = vec.fit_transform(train_data.tweet)
test_tfidf = vec.transform(test_data.tweet)



In [103]:
train_tfidf.shape

(42364, 10000)

In [104]:
len(vec.vocabulary_)

10000

In [105]:
clf = naive_bayes.MultinomialNB()
clf.fit(train_tfidf,train_data["class"])
predictions = clf.predict(test_tfidf)


In [106]:
metrics.f1_score(test_data['class'],predictions)

0.8971119133574007

In [83]:
%run ../src/processing/html_tag_remover.py

In [107]:
train_data["tweet"] = train_data["tweet"].progress_apply(lambda x: strip_tags(x))
test_data["tweet"] = test_data["tweet"].progress_apply(lambda x: strip_tags(x)) 

In [108]:
vec = TfidfVectorizer(ngram_range=(1,2),max_features=10000)
train_tfidf = vec.fit_transform(train_data.tweet)
test_tfidf = vec.transform(test_data.tweet)
clf.fit(train_tfidf,train_data["class"])
predictions = clf.predict(test_tfidf)
metrics.f1_score(test_data['class'],predictions)

0.896964856230032

In [109]:
train_data["tweet"] = train_data["tweet"].progress_apply(lambda comment: re.sub(r"\s+"," ", comment))
test_data["tweet"] = test_data["tweet"].progress_apply(lambda comment: re.sub(r"\s+"," ", comment))



In [25]:
%run ../src/processing/contractions.py

talk to you later wait


In [110]:
train_data['tweet'] = train_data['tweet'].progress_apply(lambda x: replace_slang(x))
test_data['tweet'] = test_data['tweet'].progress_apply(lambda x: replace_slang(x))


In [111]:
train_data['tweet'] = train_data['tweet'].progress_apply(lambda x: clean_contractions(x))
test_data['tweet'] = test_data['tweet'].progress_apply(lambda x: clean_contractions(x))


In [112]:

train_data["tweet"] = train_data["tweet"].progress_apply(lambda comment: re.sub(r"\s+"," ", comment))
test_data["tweet"] = test_data["tweet"].progress_apply(lambda comment: re.sub(r"\s+"," ", comment))


In [113]:
vec = TfidfVectorizer(ngram_range=(1,2),max_features=10000)
train_tfidf = vec.fit_transform(train_data.tweet)
test_tfidf = vec.transform(test_data.tweet)
clf.fit(train_tfidf,train_data["class"])
predictions = clf.predict(test_tfidf)
metrics.f1_score(test_data['class'],predictions)

0.8923352221890843

In [114]:
%run ../src/processing/glove_utils.py

In [115]:
word,frequency = get_top_n_words(25,vec,train_tfidf)

In [1355]:
#word

In [116]:
%run ../src/processing/regular_expressions.py

 <hashtags>: what do you… … call a pig with three eyes???piiig!!!allcaps <allcaps>  .... full time time


In [117]:
train_data['tweet'] = train_data['tweet'].progress_apply(lambda x: tweet_preprocessing(x))
test_data['tweet'] = test_data['tweet'].progress_apply(lambda x: tweet_preprocessing(x))


In [118]:

train_data["tweet"] = train_data["tweet"].progress_apply(lambda comment: re.sub(r"\s+"," ", comment))
test_data["tweet"] = test_data["tweet"].progress_apply(lambda comment: re.sub(r"\s+"," ", comment))



# some features

In [119]:
train_data["hashtags"] = train_data["tweet"].progress_apply(lambda comment: comment.count('<hashtags>'))
train_data["user"] = train_data["tweet"].progress_apply(lambda comment: comment.count("<user>"))
train_data["allcaps"]  = train_data["tweet"].progress_apply(lambda comment: comment.count("<allcaps>"))


In [120]:
test_data["hashtags"] = test_data["tweet"].progress_apply(lambda comment: comment.count('<hashtags>'))
test_data["user"] = test_data["tweet"].progress_apply(lambda comment: comment.count("<user>"))
test_data["allcaps"]  = test_data["tweet"].progress_apply(lambda comment: comment.count("<allcaps>"))



In [121]:
train_data["tweet"] = train_data["tweet"].progress_apply(lambda comment: re.sub(r"(<hashtags>|<user>|<allcaps>)"," ", comment))
test_data["tweet"] = test_data["tweet"].progress_apply(lambda comment: re.sub(r"(<hashtags>|<user>|<allcaps>)"," ", comment))

In [122]:

train_data["tweet"] = train_data["tweet"].progress_apply(lambda comment: re.sub(r"\s+"," ", comment))
test_data["tweet"] = test_data["tweet"].progress_apply(lambda comment: re.sub(r"\s+"," ", comment))



In [125]:
vec = TfidfVectorizer(ngram_range=(1,2),max_features=10000)
train_tfidf = vec.fit_transform(train_data.tweet)
test_tfidf = vec.transform(test_data.tweet)
clf.fit(train_tfidf,train_data["class"])
predictions = clf.predict(test_tfidf)
metrics.f1_score(test_data['class'],predictions)

0.8811160533764658

In [124]:
word,frequency = get_top_n_words(25,vec,train_tfidf)

In [128]:
stopwords = ['the','to','and','is','my','just','us','its','it','up','in','for','of','not','at','this','on','am','that','be','with','are','me','it','all','like','so','do','rt','ff','you','your','if','as','or','an','a','i','f','it','day','but','have','about',"we","after","who","what","day","when","such","how","by","been","she","he","their","wa","today","from","then","me","via","only","off","now","let","our","by","new","many","only","there","here","him","her","today","one","has","d","s","h","a","o","f"]

In [129]:
def count_stop_words(text):
    count=0
    for word in text.lower().split():
        if word in stopwords:
            count+=1
    return count

In [130]:
train_data['stop_word_count'] = train_data['tweet'].progress_apply(lambda x: count_stop_words(x))
test_data['stop_word_count'] = test_data['tweet'].progress_apply(lambda x: count_stop_words(x))


In [131]:
train_data['tweet'] = train_data['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stopwords)]))
test_data['tweet'] = test_data['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stopwords)]))


In [132]:

train_data["tweet"] = train_data["tweet"].progress_apply(lambda comment: re.sub(r"\s+"," ", comment))
test_data["tweet"] = test_data["tweet"].progress_apply(lambda comment: re.sub(r"\s+"," ", comment))


In [137]:
drop_rows = train_data[train_data['tweet']==""]
train_data.drop(drop_rows.index, axis=0,inplace=True)

In [138]:
drop_rows = test_data[test_data['tweet']==""]
test_data.drop(drop_rows.index, axis=0,inplace=True)

train_data["tweet"] = train_data["tweet"].progress_apply(lambda comment: re.sub(r"\s+"," ", comment))
test_data["tweet"] = test_data["tweet"].progress_apply(lambda comment: re.sub(r"\s+"," ", comment))


In [139]:
%run ../src/processing/spellings.py

In [140]:
train_data['tweet'] = train_data['tweet'].progress_apply(lambda x: replace_elongated_text(x))
test_data['tweet'] = test_data['tweet'].progress_apply(lambda x: replace_elongated_text(x))



In [141]:
vec = TfidfVectorizer(ngram_range=(1,2),max_features=10000)
train_tfidf = vec.fit_transform(train_data.tweet)
test_tfidf = vec.transform(test_data.tweet)
clf.fit(train_tfidf,train_data["class"])
predictions = clf.predict(test_tfidf)
metrics.f1_score(test_data['class'],predictions)

0.8870918210666394

In [142]:
word,frequency = get_top_n_words(25,vec,train_tfidf)

In [108]:
#word

In [143]:
%run ../src/processing/emojis.py


smiling face fmgvmn steaming bowl  sleeping face  exploding head  woozy face fgfb thinking face


In [144]:
train_data['tweet'] = train_data['tweet'].progress_apply(lambda x: replace_emojis_with_text(x))
test_data['tweet'] = test_data['tweet'].progress_apply(lambda x: replace_emojis_with_text(x))



In [145]:
vec = TfidfVectorizer(ngram_range=(1,2),max_features=10000)
train_tfidf = vec.fit_transform(train_data.tweet)
test_tfidf = vec.transform(test_data.tweet)
clf.fit(train_tfidf,train_data["class"])
predictions = clf.predict(test_tfidf)
metrics.f1_score(test_data['class'],predictions)

0.8822803045894217

In [146]:
train_data['num_exclamation_marks'] = train_data['tweet'].progress_apply(lambda comment: comment.count('!'))
train_data['num_question_marks'] = train_data['tweet'].progress_apply(lambda comment: comment.count('?'))
train_data['num_punctuation'] = train_data['tweet'].progress_apply(lambda comment: sum(comment.count(w) for w in '.,;:'))
train_data['num_symbols'] = train_data['tweet'].progress_apply(lambda comment: sum(comment.count(w) for w in '*&$%'))

In [147]:
test_data['num_exclamation_marks'] = test_data['tweet'].progress_apply(lambda comment: comment.count('!'))
test_data['num_question_marks'] = test_data['tweet'].progress_apply(lambda comment: comment.count('?'))
test_data['num_punctuation'] = test_data['tweet'].progress_apply(lambda comment: sum(comment.count(w) for w in '.,;:'))
test_data['num_symbols'] = test_data['tweet'].progress_apply(lambda comment: sum(comment.count(w) for w in '*&$%'))

In [56]:
%run ../src/processing/special_char.py

dfvfe ffve  rfvfe s d v          eee     


In [148]:
train_data['tweet'] = train_data['tweet'].progress_apply(lambda x: clean_special_chars(x))
test_data['tweet'] = test_data['tweet'].progress_apply(lambda x: clean_special_chars(x))



In [167]:
train_data['total_length'] = train_data['tweet'].progress_apply(len)
train_data['capitals'] = train_data['tweet'].progress_apply(lambda comment: sum(1 for c in comment if c.isupper()))
train_data['caps_vs_length'] =  train_data['capitals']/train_data['total_length']
train_data['num_words'] = train_data['tweet'].progress_apply(lambda comment: len(comment.split()))
train_data['num_unique_words'] = train_data['tweet'].progress_apply(lambda comment: len(set(w for w in comment.split())))
train_data['words_vs_unique'] = train_data['num_unique_words'] / train_data['num_words']

In [166]:
test_data['total_length'] = test_data['tweet'].progress_apply(len)
test_data['capitals'] = test_data['tweet'].progress_apply(lambda comment: sum(1 for c in comment if c.isupper()))
test_data['caps_vs_length'] =  test_data['capitals']/test_data['total_length']
test_data['num_words'] = test_data['tweet'].progress_apply(lambda comment: len(comment.split()))
test_data['num_unique_words'] = test_data['tweet'].progress_apply(lambda comment: len(set(w for w in comment.split())))
test_data['words_vs_unique'] = test_data['num_unique_words'] / train_data['num_words']



In [168]:
vec = TfidfVectorizer(ngram_range=(1,2),max_features=10000)
train_tfidf = vec.fit_transform(train_data.tweet)
test_tfidf = vec.transform(test_data.tweet)
clf.fit(train_tfidf,train_data["class"])
predictions = clf.predict(test_tfidf)
metrics.f1_score(test_data['class'],predictions)

0.8815478028198004

In [181]:

train_data["tweet"] = train_data["tweet"].apply(lambda x: re.sub(r"['>/]"," ",x))
train_data["tweet"] = train_data["tweet"].apply(lambda x: re.sub(r"[']"," ",x))
train_data["tweet"] = train_data["tweet"].apply(lambda x: re.sub(r"(\.+)"," ",x))
train_data["tweet"] = train_data["tweet"].apply(lambda x: re.sub(r"(\s+)"," ",x))
train_data["tweet"] = train_data["tweet"].apply(lambda x: x.strip())

In [182]:

test_data["tweet"] = test_data["tweet"].apply(lambda x: re.sub(r"['>/]"," ",x))
test_data["tweet"] = test_data["tweet"].apply(lambda x: re.sub(r"(\.+)"," ",x))
test_data["tweet"] = test_data["tweet"].apply(lambda x: re.sub(r"[']"," ",x))
test_data["tweet"] = test_data["tweet"].apply(lambda x: re.sub(r"(\s+)"," ",x))
test_data["tweet"] = test_data["tweet"].apply(lambda x: x.strip())

In [360]:
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
#stemming decreased f1 score

In [65]:
train_data.shape
train_data.columns

Index(['tweet', 'class', 'hashtags', 'user', 'allcaps', 'stop_word_count',
       'num_exclamation_marks', 'num_question_marks', 'num_punctuation',
       'num_symbols', 'total_length', 'capitals', 'caps_vs_length',
       'num_words', 'num_unique_words', 'words_vs_unique'],
      dtype='object')

In [163]:
#total length is not 0 but num_words is, its quite evident that they all contains spaces
len(train_data[train_data.num_words==0].tweet)

6

In [164]:
drop_rows = train_data[train_data['num_words']==0]
train_data.drop(drop_rows.index, axis=0,inplace=True)

In [184]:
len(train_data[train_data.num_words==0].tweet)

0

In [171]:
drop_rows = test_data[test_data['num_words']==0]
test_data.drop(drop_rows.index, axis=0,inplace=True)

In [179]:
train_data["tweet"] = train_data["tweet"].apply(lambda x: x.strip())
test_data["tweet"] = test_data["tweet"].apply(lambda x: x.strip())

In [185]:
train_data.to_csv(Path('../dataset/ML_CLEANED_train_data'))
test_data.to_csv(Path('../dataset/ML_CLEANED_test_data'))


In [187]:
pd.read_csv(CONST.CLEANED_TEST).shape

(11882, 17)